## Import packages

Note that `rasterio` and `geopandas` need to be installed in the current Python environment for this notebook to work.

In [ ]:
import cv2
from keras.utils import load_img
from keras.saving import load_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from segment_anything import sam_model_registry, SamPredictor
from skimage.measure import regionprops, regionprops_table
from tqdm import trange, tqdm

import segmenteverygrain as seg

%matplotlib qt

## Load models

In [ ]:
# UNET model
unet = load_model(
    "./models/seg_model.keras",
    custom_objects={'weighted_crossentropy': seg.weighted_crossentropy})

# SAM checkpoints. Download from:
# https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](
    checkpoint="./models/sam_vit_h_4b8939.pth")

/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = tor

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

In [ ]:
fname = './examples/RI_T01_Grid_65/RI_T01_Grid_65.tif'
image = np.array(load_img(fname))
image_pred = seg.predict_image(image, unet, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts 
# (and longer processing times):
labels, coords = seg.label_grains(image, image_pred, dbs_max_dist=10.0)

# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(
    sam, image, image_pred, coords, labels, 
    min_area=50.0, plot_image=True, 
    remove_edge_grains=False, remove_large_objects=False)

segmenting image tiles...


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  5.58it/s]


creating masks using SAM...


100%|███████████████████████████████████████| 1301/1301 [00:52<00:00, 24.61it/s]


finding overlapping polygons...


1293it [00:03, 362.92it/s]


finding best polygons...


100%|█████████████████████████████████████████| 393/393 [00:08<00:00, 46.56it/s]


creating labeled image...


100%|████████████████████████████████████████| 428/428 [00:01<00:00, 310.75it/s]


## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [ ]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
    lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
    lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [41]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [42]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 426/426 [00:00<00:00, 2442.87it/s]
426it [00:00, 3401.71it/s]


Plot the updated set of grains:

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(image)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(
    image, all_grains, ax, cmap='Paired')
seg.plot_grain_axes_and_centroids(
    all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0]);

100%|████████████████████████████████████████| 426/426 [00:01<00:00, 331.40it/s]


## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [ ]:
predictor = SamPredictor(sam)
predictor.set_image(image) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', 
    lambda event: seg.onclick(event, ax, coords, image, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event',
    lambda event: seg.onpress(event, ax, fig))

In [45]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [46]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 451/451 [00:00<00:00, 2533.99it/s]
451it [00:00, 3316.39it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [ ]:
cid5 = fig.canvas.mpl_connect('button_press_event',
    lambda event: seg.click_for_scale(event, ax))

number of pixels: 193.11


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [ ]:
n_of_units = 500 # centimeters in the case of 'IMG_5208_image.png'
units_per_pixel = n_of_units / 193.11 # length of scale bar in pixels

In [49]:
props = regionprops_table(labels.astype('int'), intensity_image = image, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [50]:
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,4578.794839,182.021962,1097.698389,97.993063,63.239928,1.063560,272.525592,251.0,252.0,249.0,141.670571,137.970717,132.992679,0.0,0.0,0.0
1,2,4531.867220,189.627219,1041.396450,96.602816,60.762530,-1.549856,261.540555,254.0,255.0,255.0,163.689349,162.676036,172.184911,18.0,13.0,17.0
2,3,2748.617693,255.880488,719.348780,77.825721,46.443157,-1.490290,206.983392,253.0,255.0,255.0,184.565854,184.504878,188.229268,25.0,20.0,28.0
3,4,2634.650618,348.519084,1015.414758,75.733510,51.466855,-1.545797,231.726671,244.0,242.0,240.0,122.595420,117.417303,113.127226,17.0,7.0,6.0
4,5,2373.196740,587.093220,607.022599,64.949000,47.262029,-0.580464,183.236375,253.0,255.0,255.0,133.307910,132.861582,150.985876,12.0,11.0,34.0


In [55]:
# plot histogram of grain axis lengths
# note that input data needs to be in milimeters
fig, ax = seg.plot_histogram_of_axis_lengths(grain_data['major_axis_length'], grain_data['minor_axis_length'], binsize=0.2)

## Save mask and image to PNG files

In [ ]:
dirname = './examples/RI_T01_Grid_65/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png',
    cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True

## Convert polygon row, col coordinates to projected coordinates and save them to shapefile

In [56]:
import rasterio
dataset = rasterio.open(fname)

In [ ]:
# convert polygon coordinates from row, col to UTM
from shapely.geometry import Polygon
projected_polys = []
for grain in all_grains:
    x, y = rasterio.transform.xy(
        dataset.transform, grain.exterior.xy[1], grain.exterior.xy[0])
    poly = Polygon(np.vstack((x, y)).T)
    projected_polys.append(poly)

In [59]:
# create geopandas dataframe
import geopandas
gdf = geopandas.GeoDataFrame(projected_polys, columns = ['geometry'])
gdf.head()

,geometry
0,"POLYGON ((339043.943 4686358.495, 339043.941 4..."
1,"POLYGON ((339043.824 4686358.479, 339043.822 4..."
2,"POLYGON ((339043.228 4686358.364, 339043.226 4..."
3,"POLYGON ((339043.768 4686358.191, 339043.767 4..."
4,"POLYGON ((339043.036 4686357.764, 339043.034 4..."


In [ ]:
# create property dataframe from labeled image
props = regionprops_table(
    labels.astype('int'), 
    intensity_image = image, 
    properties = ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values
grain_data['area'] = grain_data['area'].values
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length
0,1,683.0,182.021962,1097.698389,37.846881,24.424525
1,2,676.0,189.627219,1041.396450,37.309940,23.467704
2,3,410.0,255.880488,719.348780,30.057850,17.937276
3,4,393.0,348.519084,1015.414758,29.249796,19.877529
4,5,354.0,587.093220,607.022599,25.084603,18.253541


In [ ]:
# convert centroids from row, col to UTM and add them to geodataframe
centroid_x, centroid_y = rasterio.transform.xy(
    dataset.transform, grain_data['centroid-0'], grain_data['centroid-1'])
gdf['centroid_x'] = centroid_x
gdf['centroid_y'] = centroid_y

In [62]:
# convert grain axis lengths to UTM units
gdf['major_axis_length'] = grain_data['major_axis_length'] * dataset.transform[0]
gdf['minor_axis_length'] = grain_data['minor_axis_length'] * dataset.transform[0]
gdf.head()

,geometry,centroid_x,centroid_y,major_axis_length,minor_axis_length
0,"POLYGON ((339043.943 4686358.495, 339043.941 4...",339043.917150,4.686359e+06,0.068124,0.043964
1,"POLYGON ((339043.824 4686358.479, 339043.822 4...",339043.815807,4.686359e+06,0.067158,0.042242
2,"POLYGON ((339043.228 4686358.364, 339043.226 4...",339043.236121,4.686358e+06,0.054104,0.032287
3,"POLYGON ((339043.768 4686358.191, 339043.767 4...",339043.769040,4.686358e+06,0.052650,0.035780
4,"POLYGON ((339043.036 4686357.764, 339043.034 4...",339043.033934,4.686358e+06,0.045152,0.032856


In [ ]:
# check if everything looks good
band1 = dataset.read(1)
band2 = dataset.read(2)
band3 = dataset.read(3)
plt.figure()
plt.imshow(np.stack((band1, band2, band3), axis=2),
    extent = [dataset.bounds[0], dataset.bounds[2], 
              dataset.bounds[1], dataset.bounds[3]])
plt.scatter(gdf['centroid_x'], gdf['centroid_y']);

In [ ]:
# set geodataframe CRS
gdf.crs = dataset.crs

In [ ]:
# write shapefile
gdf.to_file("./examples/RI_T01_Grid_65/projected_grains.shp")

/var/folders/wn/31h1q45j6y321jzk91br_d3m0000gn/T/ipykernel_7410/571464815.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("/Users/zoltan/Dropbox/Segmentation/Kalinda_Roberts_project/projected_grains.shp") # write shapefile
/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'major_axis_length' to 'major_axis'
  ogr_write(
/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'minor_axis_length' to 'minor_axis'
  ogr_write(


In [66]:
dataset.close()